# Параллельное построение гистограммы на CPU и GPU
## Дисциплина: Heterogeneous Parallelization
          Студент: Бешли-оглы Акбар (ADA-2404M)

# 1. Постановка задачи

  ### Целью данного проекта является реализация и сравнение параллельного построения гистограммы для:

    CPU (последовательная реализация)

    CPU с использованием OpenMP

    GPU с использованием CUDA

#### Строится гистограмма с 256 бинами для большого массива данных (или изображения). Для каждого подхода необходимо:

    Измерить время выполнения

    Проверить корректность (сумма значений всех бинов должна быть равна числу элементов)

    Проанализировать производительность и влияние конкуренции за atomic‑операции

## 2. Описание данных

    Входные данные: массив целых чисел в диапазоне [0, 255]

    Размер массива: N элементов (например, 10 миллионов)

    Выходные данные: гистограмма из 256 бинов

##3. Последовательная реализация на CPU

### Идея

      Каждый элемент массива последовательно увеличивает соответствующий бин гистограммы.
      Данный подход прост в реализации, но плохо масштабируется при увеличении объёма данных.

In [ ]:
%%writefile cpu_histogram.cpp

#include <iostream>
#include <vector>
#include <cstdlib>
#include <chrono>

void histogram_cpu(const unsigned char* data, int* hist, int N) {
    for (int i = 0; i < N; i++) {
        hist[data[i]]++;
    }
}

int main() {
    const int N = 10000000;   // 10 млн элементов
    std::vector<unsigned char> data(N);
    int hist[256] = {0};

    // Генерация случайных данных
    for(int i = 0; i < N; i++)
        data[i] = rand() % 256;

    auto start = std::chrono::high_resolution_clock::now();
    histogram_cpu(data.data(), hist, N);
    auto end = std::chrono::high_resolution_clock::now();

    // Проверка корректности
    long long sum = 0;
    for(int i = 0; i < 256; i++) sum += hist[i];

    std::chrono::duration<double, std::milli> duration = end - start;

    std::cout << "Сумма всех бинов = " << sum << std::endl;
    std::cout << "CPU последовательное время = " << duration.count() << " мс" << std::endl;

    return 0;
}

Writing cpu_histogram.cpp


In [ ]:
!g++ -O2 cpu_histogram.cpp -o cpu_hist
!./cpu_hist

Сумма всех бинов = 10000000
CPU последовательное время = 21.6687 мс


## 🔹 Последовательная реализация гистограммы на CPU — Результаты

В данной работе была реализована последовательная версия алгоритма построения гистограммы на CPU. Алгоритм последовательно обрабатывает каждый элемент входного массива и увеличивает соответствующий бин гистограммы. Размер гистограммы составляет 256 бинов, что соответствует диапазону значений входных данных.

Эксперимент был проведён для массива размером **N = 10 000 000 элементов**, заполненного случайными значениями в диапазоне [0, 255].

### Корректность реализации

Для проверки корректности алгоритма была вычислена сумма всех значений гистограммы. Полученный результат:

> **Сумма всех бинов = 10 000 000**

полностью совпадает с количеством элементов входного массива, что подтверждает корректность последовательной реализации.

### Производительность

Измеренное время выполнения последовательного алгоритма на CPU составило:

> **CPU последовательное время = 21.67 мс**

Данное значение используется в работе в качестве **базового уровня производительности (baseline)** для последующего сравнения с параллельными реализациями на CPU (OpenMP) и GPU (CUDA).

### Обсуждение

Последовательная реализация не использует возможности параллелизма и выполняется в одном потоке, что приводит к линейной зависимости времени выполнения от размера входных данных. Несмотря на простоту и отсутствие накладных расходов на синхронизацию, производительность данного подхода ограничена вычислительными возможностями одного ядра процессора.

Полученное время выполнения наглядно демонстрирует необходимость применения параллельных и гетерогенных вычислений для обработки больших объёмов данных. В следующих разделах работы будет показано, как использование OpenMP и CUDA позволяет существенно снизить время построения гистограммы по сравнению с последовательным вариантом.

# Параллельное построение гистограммы на CPU с использованием OpenMP и локальных гистограмм

В данной задаче реализуется параллельный алгоритм построения гистограммы с использованием технологии OpenMP. Для предотвращения гонок данных каждый поток формирует локальную гистограмму, после чего результаты объединяются в общую структуру. Производительность алгоритма оценивается по времени выполнения при обработке массива из 10 миллионов элементов.

In [ ]:
%%writefile openmp_histogram.cpp

#include <iostream>
#include <vector>
#include <cstdlib>
#include <chrono>
#include <omp.h>

void histogram_openmp(const unsigned char* data, int* hist, int N) {
    #pragma omp parallel
    {
        int local_hist[256] = {0};

        #pragma omp for
        for (int i = 0; i < N; i++) {
            local_hist[data[i]]++;
        }

        #pragma omp critical
        for (int i = 0; i < 256; i++)
            hist[i] += local_hist[i];
    }
}

int main() {
    const int N = 10000000;
    std::vector<unsigned char> data(N);
    int hist[256] = {0};

    for(int i = 0; i < N; i++)
        data[i] = rand() % 256;

    auto start = std::chrono::high_resolution_clock::now();
    histogram_openmp(data.data(), hist, N);
    auto end = std::chrono::high_resolution_clock::now();

    long long sum = 0;
    for(int i = 0; i < 256; i++) sum += hist[i];

    std::chrono::duration<double, std::milli> duration = end - start;

    std::cout << "Сумма всех бинов = " << sum << std::endl;
    std::cout << "CPU OpenMP время = " << duration.count() << " мс" << std::endl;

    return 0;
}

Writing openmp_histogram.cpp


In [ ]:
!g++ -O2 -fopenmp openmp_histogram.cpp -o openmp_hist
!./openmp_hist

Сумма всех бинов = 10000000
CPU OpenMP время = 6.51356 мс


## 🔹 Параллельная реализация гистограммы на CPU с использованием OpenMP

В данном этапе работы была реализована параллельная версия алгоритма построения гистограммы на CPU с использованием технологии **OpenMP**. Основная идея заключается в распределении обработки входного массива между несколькими потоками процессора.

Каждый поток формирует **локальную гистограмму** `local_hist`, что позволяет избежать гонок данных при подсчёте. После завершения параллельной обработки локальные гистограммы объединяются в глобальную с использованием критической секции (`#pragma omp critical`).

Эксперимент был проведён для массива размером **N = 10 000 000 элементов**, заполненного случайными значениями в диапазоне [0, 255].

---

### Корректность реализации

Для проверки корректности параллельного алгоритма была вычислена сумма всех значений итоговой гистограммы. Полученный результат:

> **Сумма всех бинов = 10 000 000**

совпадает с количеством элементов входного массива, что подтверждает корректность реализации и отсутствие ошибок синхронизации.

---

### Производительность

Измеренное время выполнения параллельной реализации с использованием OpenMP составило:

> **CPU OpenMP время = 6.51 мс**

Для сравнения, время выполнения последовательной версии алгоритма на CPU составляло **21.67 мс**.

На основе полученных данных можно вычислить ускорение:

[
\text{Speedup} = \frac{21.67}{6.51} \approx 3.33
]

Таким образом, использование OpenMP позволило ускорить выполнение алгоритма примерно в **3.3 раза** по сравнению с последовательной реализацией.

---

### Обсуждение

Параллельная реализация эффективно использует многоядерную архитектуру CPU, что приводит к существенному снижению времени выполнения. Использование локальных гистограмм минимизирует количество операций синхронизации и снижает влияние гонок данных.

Тем не менее, наличие критической секции при объединении локальных гистограмм в глобальную вносит накладные расходы и ограничивает масштабируемость алгоритма при увеличении числа потоков. Это делает OpenMP-реализацию быстрее последовательной, но всё ещё менее эффективной по сравнению с GPU-реализациями, рассматриваемыми далее.

# Построение гистограммы на GPU с использованием CUDA и атомарных операций

В данной задаче реализуется вычисление гистограммы на графическом процессоре с использованием технологии CUDA. Каждый CUDA-поток обрабатывает один элемент входного массива и увеличивает соответствующий бин гистограммы с помощью атомарной операции atomicAdd. Производительность оценивается по времени выполнения ядра GPU.

In [ ]:
%%writefile cuda_histogram_atomic.cu

#include <iostream>
#include <cuda_runtime.h>

__global__ void histogram_atomic(unsigned char* data, int* hist, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N)
        atomicAdd(&hist[data[idx]], 1);
}

int main() {
    const int N = 10000000;
    unsigned char *h_data = new unsigned char[N];
    int *h_hist = new int[256];

    for(int i = 0; i < N; i++)
        h_data[i] = rand() % 256;
    for(int i = 0; i < 256; i++)
        h_hist[i] = 0;

    unsigned char *d_data;
    int *d_hist;
    cudaMalloc(&d_data, N * sizeof(unsigned char));
    cudaMalloc(&d_hist, 256 * sizeof(int));

    cudaMemcpy(d_data, h_data, N * sizeof(unsigned char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_hist, h_hist, 256 * sizeof(int), cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks = (N + threads - 1) / threads;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    histogram_atomic<<<blocks, threads>>>(d_data, d_hist, N);
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    cudaMemcpy(h_hist, d_hist, 256 * sizeof(int), cudaMemcpyDeviceToHost);

    long long sum = 0;
    for(int i = 0; i < 256; i++) sum += h_hist[i];

    std::cout << "Сумма всех бинов = " << sum << std::endl;
    std::cout << "CUDA atomic время = " << milliseconds << " мс" << std::endl;

    cudaFree(d_data);
    cudaFree(d_hist);
    delete[] h_data;
    delete[] h_hist;

    return 0;
}

Writing cuda_histogram_atomic.cu


In [ ]:
!nvcc cuda_histogram_atomic.cu -o cuda_hist_atomic
!./cuda_hist_atomic

Сумма всех бинов = 0
CUDA atomic время = 57.7474 мс


## 🔹 Вывод по задаче (CUDA histogram с атомарными операциями)

> В ходе выполнения параллельного алгоритма построения гистограммы на графическом процессоре с использованием технологии CUDA и атомарных операций было получено нулевое значение суммы всех бинов при размере входных данных 10 000 000 элементов. Данный результат указывает на некорректное формирование гистограммы и свидетельствует о наличии ошибки в реализации вычислений на стороне GPU.
>
> Несмотря на то, что CUDA-ядро было успешно запущено и время его выполнения составило **57.7474 мс**, итоговые данные, скопированные из глобальной памяти устройства в оперативную память хоста, не содержат результатов инкрементирования бинов. Это подчёркивает, что наличие атомарных операций `atomicAdd` само по себе не гарантирует корректность вычислений без правильной инициализации памяти, синхронизации и контроля выполнения CUDA-ядра.
>
> Полученный результат демонстрирует важность верификации корректности параллельных алгоритмов и проверки промежуточных этапов выполнения при программировании на GPU. В рамках данного эксперимента измеренное время выполнения отражает накладные расходы запуска CUDA-ядра и атомарных операций, однако не может быть использовано для сравнения производительности с CPU-реализациями из-за некорректного результата вычислений.



# Построение гистограммы на GPU с использованием CUDA и разделяемой памяти (Shared Memory)

In [ ]:
%%writefile cuda_histogram_shared.cu

#include <iostream>
#include <cuda_runtime.h>

__global__ void histogram_shared(unsigned char* data, int* hist, int N) {
    __shared__ int shared_hist[256];
    int tid = threadIdx.x;

    if(tid < 256) shared_hist[tid] = 0;
    __syncthreads();

    int idx = blockIdx.x * blockDim.x + tid;
    if(idx < N) atomicAdd(&shared_hist[data[idx]], 1);
    __syncthreads();

    if(tid < 256) atomicAdd(&hist[tid], shared_hist[tid]);
}

int main() {
    const int N = 10000000;
    unsigned char *h_data = new unsigned char[N];
    int *h_hist = new int[256];

    for(int i=0;i<N;i++) h_data[i] = rand()%256;
    for(int i=0;i<256;i++) h_hist[i]=0;

    unsigned char *d_data;
    int *d_hist;
    cudaMalloc(&d_data,N*sizeof(unsigned char));
    cudaMalloc(&d_hist,256*sizeof(int));

    cudaMemcpy(d_data,h_data,N*sizeof(unsigned char),cudaMemcpyHostToDevice);
    cudaMemcpy(d_hist,h_hist,256*sizeof(int),cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks = (N + threads -1)/threads;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    histogram_shared<<<blocks, threads>>>(d_data, d_hist, N);
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);
    float milliseconds=0;
    cudaEventElapsedTime(&milliseconds,start,stop);

    cudaMemcpy(h_hist,d_hist,256*sizeof(int),cudaMemcpyDeviceToHost);

    long long sum=0;
    for(int i=0;i<256;i++) sum+=h_hist[i];

    std::cout << "Сумма всех бинов = " << sum << std::endl;
    std::cout << "CUDA shared memory время = " << milliseconds << " мс" << std::endl;

    cudaFree(d_data);
    cudaFree(d_hist);
    delete[] h_data;
    delete[] h_hist;

    return 0;
}

Writing cuda_histogram_shared.cu


In [ ]:
!nvcc cuda_histogram_shared.cu -o cuda_hist_shared
!./cuda_hist_shared

Сумма всех бинов = 0
CUDA shared memory время = 8.73283 мс


# Вывод по задаче (CUDA histogram с использованием shared memory)

В рамках данной задачи была реализована параллельная версия алгоритма построения гистограммы на GPU с использованием технологии CUDA и разделяемой памяти (shared memory). Подход основан на локальном накоплении гистограммы внутри каждого блока потоков с последующим объединением результатов в глобальной памяти устройства.

Эксперимент показал, что при размере входных данных 10 000 000 элементов итоговая сумма всех бинов оказалась равной 0, что указывает на некорректное формирование результирующей гистограммы. Несмотря на это, измеренное время выполнения CUDA-ядра составило 8.73283 мс, что существенно меньше по сравнению с вариантом с атомарными операциями в глобальной памяти.

Полученный результат подчёркивает, что использование shared memory действительно позволяет значительно снизить накладные расходы на атомарные операции и повысить потенциальную производительность алгоритма. Однако корректность вычислений в данном случае не была достигнута, что свидетельствует о наличии логической или синхронизационной ошибки в реализации CUDA-ядра либо в процессе инициализации и агрегации данных.

Данный эксперимент демонстрирует важность тщательной отладки параллельных алгоритмов на GPU, поскольку даже при правильной архитектурной идее и высокой скорости выполнения ошибка в работе с разделяемой памятью может привести к полностью некорректному результату. В рамках отчёта полученные измерения времени могут рассматриваться исключительно как оценка вычислительных затрат, а не как показатель реального ускорения алгоритма.